In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = ('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

batch_size=32
lr=3e-4
epoch=10

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform= transform)
trainload = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
testload = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=0)

class bottleneck(nn.Module):
  def __init__(self, inpc, outc, stride=1, downsample=None):
    super(bottleneck,self).__init__()
    self.stride = stride
    self.conv1 = nn.Conv2d(inpc, outc, 1, stride=1, padding=0)
    self.conv2 = nn.Conv2d(outc, outc, 3, stride=stride, padding=1)       
    self.conv3 = nn.Conv2d(outc, outc*4, 1, stride=1, padding=0)
    self.bn1 = nn.BatchNorm2d(outc)
    self.bn2 = nn.BatchNorm2d(outc*4)
    self.ReLU = nn.ReLU()
    self.downsample = downsample
  def forward(self, x):
    identity=x
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.conv2(x)
    x = self.bn1(x)
    x = self.conv3(x)
    x = self.bn2(x)
    if(self.downsample):
      identity = self.downsample(identity)          #in order to make the dimensions same(which is reqd for addition)
    x+=identity                                    #adding the values of the input
    x = self.ReLU(x)
    return x

class residualblock(nn.Module):
  def __init__(self, block, layer, noclasses):
    super(residualblock, self).__init__()
    self.inchannels = 64
    self.conv1 = nn.Conv2d(3, self.inchannels, 7, stride = 2, padding = 3)
    self.bn1 = nn.BatchNorm2d(self.inchannels)
    self.maxpool = nn.MaxPool2d(3, stride = 2, padding = 1)             
    self.ReLU = nn.ReLU()
    self.layer1 = self.make_layer(bottleneck, 64, layer[0], stride = 1)
    self.layer2 = self.make_layer(bottleneck, 128, layer[1], stride = 2)
    self.layer3 = self.make_layer(bottleneck, 256, layer[2], stride = 2)  
    self.layer4 = self.make_layer(bottleneck, 512, layer[3], stride = 2)
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(512*4 , noclasses)
    self.softmax = nn.Softmax(dim = 1)
  def make_layer(self, block, outplanes, noblocks, stride=1):
    downsample = None
    if(stride!=1 or self.inchannels!=outplanes*4):
      downsample = nn.Sequential(
          nn.Conv2d(self.inchannels, outplanes*4 , 1, stride = stride, padding = 0),
          nn.BatchNorm2d(outplanes*4)
      )
    layers = []
    layers.append(block(self.inchannels, outplanes, stride, downsample))
    self.inchannels = outplanes*4
    for i in range(1,noblocks):
      layers.append(block(self.inchannels, outplanes))
    return nn.Sequential(*layers)     #layers is an array of commands, * returns them in order to the sequential
  def forward(self, x):
    x=self.conv1(x)
    x=self.bn1(x)
    x=self.ReLU(x)
    x=self.maxpool(x)
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)
    x=self.avgpool(x)
    x=torch.flatten(x,1)
    x=self.fc(x)
    x=self.softmax(x)
    return x

def Res50(noclasses):
  return residualblock(bottleneck,[3,6,3,4],noclasses)


mod = Res50(noclasses = 10).to(device)
lossfn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mod.parameters(), lr=lr)
for j in range(epoch):
  for image,label in trainload:
    image=image.to(device)
    label=label.to(device)
    output = mod(image)
    loss = lossfn(output,label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(j+1,"\n")

cor=0
incor=0
with torch.no_grad():
  for image,label in testload:
    image=image.to(device)
    label=label.to(device)
    output = mod(image)
    temp, pred = torch.max(output, 1)
    cor += (pred == label).sum().item()
    incor += (pred != label).sum().item()

model_accuracy = (cor/(cor + incor)) * 100
print("Accuracy of the ResNet model : ", model_accuracy, "%", sep = "")

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

Accuracy of the ResNet model : 50.4%
